# 6. Continued Feature Engineering & Feature Selection + Model Selection and Fine Tuning

In [1]:
# ===============================
# Imports & data load
# ===============================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

# Optional (installed when needed)
try:
    import category_encoders as ce
except Exception:
    !pip -q install category-encoders
    import category_encoders as ce

In [2]:
# ===============================
# Data load
# ===============================
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Spotify_cleaned_stage5.csv')
print(df.shape)
df.head()

(29729, 33)


,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,popularity_class,release_year,release_month,dance_energy,valence_energy,acoustic_instru,energy_diff,speech_loud_ratio,log_duration,release_age
0,6f807x0ima9a1j3VPbc7VN,I Dont Care with Justin Bieber Loud Luxury Remix,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Dont Care with Justin Bieber Loud Luxury Remix,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,60–80,2019,6,0.685168,0.474488,0.102000,0.8140,0.016043,12.179498,6
1,0r7CVbZTWZgbTCYdfa2P31,Memories Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories Dillon Francis Remix,2019-12-13,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,60–80,2019,12,0.591690,0.564795,0.076610,0.7426,0.006249,11.999055,6
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time Don Diablo Remix,2019-05-07,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,60–80,2019,5,0.628425,0.570703,0.079423,0.8516,0.016742,12.081739,6
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine The Remixes,2019-07-19,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,40–60,2019,7,0.667740,0.257610,0.028709,0.9013,0.021348,12.038210,6
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved Future Humans Remix,2019-05-03,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,60–80,2019,5,0.541450,0.603925,0.080300,0.7527,0.006329,12.149783,6


In [3]:
# ===============================
# Stage 5 — Enrichment (idempotent)
# ===============================
need = lambda c: c not in df.columns

# Core interaction features
if need("dance_energy") and {"danceability","energy"} <= set(df.columns):
    df["dance_energy"] = df["danceability"] * df["energy"]

if need("valence_energy") and {"valence","energy"} <= set(df.columns):
    df["valence_energy"] = df["valence"] * df["energy"]

if need("acoustic_instru") and {"acousticness","instrumentalness"} <= set(df.columns):
    df["acoustic_instru"] = df["acousticness"] + df["instrumentalness"]

if need("energy_diff") and {"energy","acousticness"} <= set(df.columns):
    df["energy_diff"] = df["energy"] - df["acousticness"]

if need("speech_loud_ratio") and {"speechiness","loudness"} <= set(df.columns):
    df["speech_loud_ratio"] = df["speechiness"] / (df["loudness"].abs() + 1)

# Skew fixes / transforms
if need("log_duration") and "duration_ms" in df.columns:
    df["log_duration"] = np.log1p(df["duration_ms"])


In [4]:
# ===============================
# Stage 5 — Feature Selection
# ===============================
from scipy.stats import spearmanr
from sklearn.feature_selection import mutual_info_regression

target = "track_popularity"
y = df[target].astype(float)

# Numeric columns only (filter scores)
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
num_cols = [c for c in num_cols if c != target]

# Spearman
spearman_rows = []
for c in num_cols:
    s = df[[c, target]].dropna()
    if len(s) > 3:
        rho, p = spearmanr(s[c], s[target])
        spearman_rows.append({"feature": c, "spearman_rho": rho, "pvalue": p})
spearman_df = pd.DataFrame(spearman_rows).sort_values("spearman_rho", key=np.abs, ascending=False)

# Mutual Information (quick proxy)
X_mi = df[num_cols].fillna(df[num_cols].median())
mi = mutual_info_regression(X_mi, y, random_state=42)
mi_df = pd.DataFrame({"feature": num_cols, "mi": mi}).sort_values("mi", ascending=False)

# Combine & score
score = (spearman_df.merge(mi_df, on="feature", how="outer")
                    .fillna(0)
                    .assign(score=lambda d: d["mi"] + d["spearman_rho"].abs()))
score = score.sort_values("score", ascending=False)

# Shortlist (top N numeric features — adjust N based on your dataset)
shortlist_num = score.head(25)["feature"].tolist()

# Keep also low/med-card categoricals for Stage 6 modeling
cat_keep = [c for c in ["playlist_genre","playlist_subgenre","release_year","release_month"] if c in df.columns]

# Save artifacts
pd.DataFrame({"feature": sorted(set(num_cols) | set(df.select_dtypes(include="object").columns))}).to_csv(
    "stage5_all_features.csv", index=False)
pd.DataFrame({"feature": shortlist_num + cat_keep}).to_csv(
    "stage5_shortlist_features.csv", index=False)

print("✅ Saved stage5_all_features.csv and stage5_shortlist_features.csv")
print("Top 10 numeric by combined score:\n", score.head(10))

✅ Saved stage5_all_features.csv and stage5_shortlist_features.csv
Top 10 numeric by combined score:
               feature  spearman_rho         pvalue        mi     score
4         duration_ms     -0.101018   2.813030e-68  0.366496  0.467513
10       log_duration     -0.101018   2.813030e-68  0.366272  0.467290
6         energy_diff     -0.139455  5.576718e-129  0.319135  0.458590
1        acousticness      0.124156  1.953468e-102  0.290673  0.414829
2        dance_energy     -0.044400   1.874017e-14  0.356165  0.400565
16  speech_loud_ratio      0.027049   3.094385e-06  0.364333  0.391382
11           loudness      0.053705   1.930545e-20  0.333236  0.386941
7    instrumentalness     -0.185204  1.247672e-227  0.199112  0.384316
18              tempo     -0.020317   4.596941e-04  0.361074  0.381390
5              energy     -0.129829  6.577832e-112  0.238473  0.368303


# Moving on to stage 6. Model Selection and Fine Tuning

In [5]:
# ===============================
# Stage 6 — Model selection (baselines + CV)
# ===============================
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Build X using shortlist + small/medium card cats
feat_cols = (shortlist_num + cat_keep +
             [c for c in ["recency_weeks","release_age","log_duration"] if c in df.columns])

X = df[feat_cols].copy()
y = df[target].astype(float)

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# --- Clean rebuild of X and preprocess using selectors ---

from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Rebuild X strictly from the final feature list (and dedupe)
feat_cols = list(dict.fromkeys(feat_cols))       # keep order, unique
X = df.loc[:, feat_cols].copy()

# Force types you want:
#    - year/month as categorical (strings)
for c in ["release_year", "release_month"]:
    if c in X.columns:
        X[c] = X[c].astype(str)

# keep these numeric (coerce if needed)
for c in ["release_age", "log_duration"]:
    if c in X.columns:
        X[c] = pd.to_numeric(X[c], errors="coerce")

# Remove any duplicate-named columns inside X (just in case)
X = X.loc[:, ~X.columns.duplicated()]

# Re-split AFTER fixing dtypes (so train/test have identical types)
from sklearn.model_selection import train_test_split
y = df["track_popularity"].astype(float)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Build preprocess with selectors (no hard-coded lists)
try:
    ohe = OneHotEncoder(handle_unknown="ignore", drop="first", sparse_output=False)
except TypeError:
    ohe = OneHotEncoder(handle_unknown="ignore", drop="first", sparse=False)

preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), selector(dtype_include=np.number)),
        ("cat", ohe, selector(dtype_include=object)),
    ],
    remainder="drop",
    verbose_feature_names_out=False
)

# sanity check: ensure selectors don't overlap
_num_cols = preprocess.transformers[0][2](X_train)
_cat_cols = preprocess.transformers[1][2](X_train)
print("Selector numeric:", list(_num_cols))
print("Selector categoricals:", list(_cat_cols))
print("Overlap?", set(_num_cols) & set(_cat_cols))


candidates = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(alpha=5.0, random_state=42),
    "Lasso": Lasso(alpha=0.1, random_state=42, max_iter=20000),
    "SVR": SVR(C=3.0, epsilon=0.5),
    "RandomForest": RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1),
    "GradientBoosting": GradientBoostingRegressor(random_state=42)
}

rows = []
for name, model in candidates.items():
    pipe = Pipeline([("prep", preprocess), ("model", model)])
    cv = cross_val_score(pipe, X_train, y_train, cv=5, scoring="r2", n_jobs=-1, error_score='raise')
    pipe.fit(X_train, y_train)
    pred = pipe.predict(X_test)
    rows.append({
        "Model": name,
        "CV_R2_mean": cv.mean(),
        "Test_R2": r2_score(y_test, pred),
        "Test_RMSE": np.sqrt(mean_squared_error(y_test, pred)),
        "Test_MAE": mean_absolute_error(y_test, pred),
    })

model_cmp = pd.DataFrame(rows).sort_values(["Test_R2","CV_R2_mean"], ascending=False)
display(model_cmp)

model_cmp.to_csv("stage6_baseline_model_comparison.csv", index=False)
print("💾 Saved → stage6_baseline_model_comparison.csv")

Selector numeric: ['duration_ms', 'log_duration', 'energy_diff', 'acousticness', 'dance_energy', 'speech_loud_ratio', 'loudness', 'instrumentalness', 'tempo', 'energy', 'valence_energy', 'acoustic_instru', 'danceability', 'release_age', 'valence', 'liveness', 'speechiness', 'key', 'mode']
Selector categoricals: ['release_year', 'release_month', 'playlist_genre', 'playlist_subgenre']
Overlap? set()


,Model,CV_R2_mean,Test_R2,Test_RMSE,Test_MAE
4,RandomForest,0.316464,0.344615,20.325869,16.271156
5,GradientBoosting,0.222558,0.222074,22.144704,18.491028
3,SVR,0.197580,0.211342,22.296936,17.951198
1,Ridge,0.207625,0.210949,22.302492,18.596466
0,LinearRegression,0.206402,0.210940,22.302615,18.587942
2,Lasso,0.180375,0.178482,22.756699,19.110996


💾 Saved → stage6_baseline_model_comparison.csv


In [7]:
from sklearn.base import BaseEstimator, TransformerMixin
import category_encoders as ce

class TEPipe(BaseEstimator, TransformerMixin):
    def __init__(self, cols=None, smoothing=0.2, min_samples_leaf=20, **kwargs):
        # store EXACTLY the init params as attributes (clone-safe)
        self.cols = cols
        self.smoothing = smoothing
        self.min_samples_leaf = min_samples_leaf
        self.kwargs = kwargs
        # internal
        self._encoder = None

    def fit(self, X, y=None):
        if not self.cols:
            return self
        self._encoder = ce.TargetEncoder(
            cols=self.cols,
            smoothing=self.smoothing,
            min_samples_leaf=self.min_samples_leaf,
            **self.kwargs
        )
        self._encoder.fit(X[self.cols], y)
        return self

    def transform(self, X):
        if not self.cols:
            return X
        Xt = X.copy()
        te_part = self._encoder.transform(X[self.cols])
        te_part.columns = self.cols  # keep names
        Xt[self.cols] = te_part
        return Xt

# choose which columns to target-encode
te_cols = [c for c in ["track_artist","playlist_genre"] if c in df.columns]

# extend features, split
feat_cols_ext = sorted(set(feat_cols + te_cols))
X = df[feat_cols_ext].copy(); y = df[target].astype(float)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# numeric & low-card categoricals (after TE)
num_feats = X_train.select_dtypes(include=[np.number]).columns.tolist()
cat_feats = X_train.select_dtypes(include="object").columns.tolist()
ohe_cols  = [c for c in cat_feats if c not in te_cols]

# OHE & scaler
try:
    ohe = OneHotEncoder(handle_unknown="ignore", drop="first", sparse_output=False)
except TypeError:
    ohe = OneHotEncoder(handle_unknown="ignore", drop="first", sparse=False)

preprocess_boost = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_feats),
        ("cat", ohe, ohe_cols),
    ],
    remainder="passthrough",
    verbose_feature_names_out=False
)

# candidates (XGB/LGBM/CatBoost)
rows = []
for name, model in candidates.items():
    te_step = TEPipe(cols=te_cols, smoothing=0.2, min_samples_leaf=20)  # fresh instance
    pipe = Pipeline([
        ("te", te_step),
        ("prep", preprocess_boost),
        ("model", model)
    ])
    cv_r2 = cross_val_score(pipe, X_train, y_train, cv=5, scoring="r2", n_jobs=-1, error_score='raise').mean()
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)

    rows.append({
        "Model": name,
        "CV_R2_mean": cv_r2,
        "Test_R2": r2_score(y_test, y_pred),
        "Test_RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
        "Test_MAE": mean_absolute_error(y_test, y_pred),
    })

boost_cmp = pd.DataFrame(rows).sort_values(["Test_R2","CV_R2_mean"], ascending=False)
display(boost_cmp)
boost_cmp.to_csv("stage6_model_comparison_boosters.csv", index=False)
print("💾 Saved → stage6_model_comparison_boosters.csv")

,Model,CV_R2_mean,Test_R2,Test_RMSE,Test_MAE
4,RandomForest,0.343490,0.378214,19.797994,15.522584
5,GradientBoosting,0.253110,0.267283,21.491599,17.790096
0,LinearRegression,0.176842,0.194388,22.535313,18.417679
1,Ridge,0.176848,0.194345,22.535919,18.419305
2,Lasso,0.167972,0.183025,22.693684,18.625824
3,SVR,0.093145,0.110265,23.682691,19.063966


💾 Saved → stage6_model_comparison_boosters.csv


In [10]:
# ===============================
# Fine-tuning
# ===============================
from sklearn.model_selection import RandomizedSearchCV

# Check if LightGBM is available
try:
    import lightgbm as lgb
    from lightgbm import LGBMRegressor
    lgb_ok = True
except ImportError:
    lgb_ok = False
    print("LightGBM not installed. Skipping LGBM fine-tuning.")

if lgb_ok:
    best = LGBMRegressor(random_state=42)
    pipe = Pipeline([("te", te_step), ("prep", preprocess_boost), ("model", best)])

    param_dist = {
        "model__n_estimators": [400, 600, 800, 1000],
        "model__learning_rate": np.linspace(0.02, 0.12, 6),
        "model__num_leaves": [31, 63, 127, 255],
        "model__min_child_samples": [10, 20, 40, 80],
        "model__subsample": np.linspace(0.6, 1.0, 5),
        "model__colsample_bytree": np.linspace(0.6, 1.0, 5),
        "model__reg_lambda": [0.0, 0.5, 1.0, 2.0],
    }

    rs = RandomizedSearchCV(
        estimator=pipe, param_distributions=param_dist,
        n_iter=25, cv=3, scoring="r2", n_jobs=-1, random_state=42, verbose=2
    )
    rs.fit(X_train, y_train)
    print("Best params:", rs.best_params_)
    pred = rs.predict(X_test)
    print(f"Tuned LGBM — Test R²: {r2_score(y_test, pred):.3f} | RMSE: {np.sqrt(mean_squared_error(y_test, pred)):.2f}")

    # Save comparison row
    tuned_row = {
        "Model": "LGBM_TUNED",
        "CV_R2_mean": rs.best_score_,
        "Test_R2": r2_score(y_test, pred),
        "Test_RMSE": np.sqrt(mean_squared_error(y_test, pred)),
        "Test_MAE": mean_absolute_error(y_test, pred),
    }
    boost_cmp = pd.concat([boost_cmp, pd.DataFrame([tuned_row])], ignore_index=True)
    boost_cmp = boost_cmp.sort_values(["Test_R2","CV_R2_mean"], ascending=False)
    display(boost_cmp)
    boost_cmp.to_csv("stage6_model_comparison_boosters.csv", index=False)

# --------------------------------------------------------------
# Save the best performing pipeline for Stage 7
# --------------------------------------------------------------

# Select the top model:
best_model_name = boost_cmp.iloc[0]["Model"]
print(f"✅ Best model from Stage 6: {best_model_name}")

# Rebuild its corresponding pipeline (same preprocessing + model)
if best_model_name == "XGB":
    from xgboost import XGBRegressor
    best_estimator = XGBRegressor(
        n_estimators=500, learning_rate=0.05, max_depth=6,
        subsample=0.8, colsample_bytree=0.8,
        reg_lambda=1.0, random_state=42, n_jobs=-1
    )
elif best_model_name == "LGBM":
    # Ensure lightgbm is imported here if LGBM was the best model
    if not lgb_ok:
        try:
            import lightgbm as lgb
            from lightgbm import LGBMRegressor
        except ImportError:
            print("Error: LightGBM is the best model but could not be imported.")
            # Handle this error, e.g., by falling back to another model or exiting.
            raise
    best_estimator = lgb.LGBMRegressor(
        n_estimators=800, learning_rate=0.05,
        subsample=0.8, colsample_bytree=0.8,
        reg_lambda=1.0, random_state=42, n_jobs=-1
    )
elif best_model_name == "LGBM_TUNED": # Handle the tuned LGBM case explicitly
    if lgb_ok and 'rs' in locals() and rs.best_estimator_ is not None:
        # Extract the model from the best pipeline found by RandomizedSearchCV
        best_estimator = rs.best_estimator_.named_steps['model']
    else:
        print("Error: Tuned LGBM was best but the RandomizedSearchCV object 'rs' or its best_estimator_ was not found.")
        raise ValueError("Could not retrieve best tuned LGBM model.")
elif best_model_name == "CatBoost":
    from catboost import CatBoostRegressor
    best_estimator = CatBoostRegressor(
        iterations=800, learning_rate=0.05, depth=8,
        loss_function="RMSE", random_state=42, verbose=False
    )
else:
    # This part handles cases where best_model_name might not be XGB, LGBM, or CatBoost
    # For instance, if the best model was from the initial 'candidates' list and not a booster
    # You might want to re-evaluate how `best_model_name` is determined or handle these explicitly.
    print(f"Warning: '{best_model_name}' is not a recognized booster for final pipeline construction. Using a placeholder or the first model from candidates.")
    best_estimator = candidates[best_model_name]

# Fresh TE and preprocessing objects
te_step = TEPipe(cols=te_cols, smoothing=0.2, min_samples_leaf=20)
best_pipe = Pipeline(
    [("te", te_step), ("prep", preprocess_boost), ("model", best_estimator)]
)

# Fit once on full training data (you can reuse X_train, y_train)
best_pipe.fit(X_train, y_train)

# Save for Stage 7
import joblib
joblib.dump(best_pipe, "best_pipe_stage6.pkl")
print("💾 Saved → best_pipe_stage6.pkl (ready for Stage 7)")

Fitting 3 folds for each of 25 candidates, totalling 75 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006241 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4556
[LightGBM] [Info] Number of data points in the train set: 23783, number of used features: 46
[LightGBM] [Info] Start training from score 43.126603
Best params: {'model__subsample': np.float64(0.6), 'model__reg_lambda': 0.0, 'model__num_leaves': 255, 'model__n_estimators': 800, 'model__min_child_samples': 40, 'model__learning_rate': np.float64(0.02), 'model__colsample_bytree': np.float64(0.6)}


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


Tuned LGBM — Test R²: 0.403 | RMSE: 19.39


,Model,CV_R2_mean,Test_R2,Test_RMSE,Test_MAE
0,LGBM_TUNED,0.343805,0.403455,19.391983,15.008485
7,LGBM_TUNED,0.343805,0.403455,19.391983,15.008485
1,RandomForest,0.343490,0.378214,19.797994,15.522584
2,GradientBoosting,0.253110,0.267283,21.491599,17.790096
3,LinearRegression,0.176842,0.194388,22.535313,18.417679
4,Ridge,0.176848,0.194345,22.535919,18.419305
5,Lasso,0.167972,0.183025,22.693684,18.625824
6,SVR,0.093145,0.110265,23.682691,19.063966


✅ Best model from Stage 6: LGBM_TUNED
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006936 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4556
[LightGBM] [Info] Number of data points in the train set: 23783, number of used features: 46
[LightGBM] [Info] Start training from score 43.126603
💾 Saved → best_pipe_stage6.pkl (ready for Stage 7)
